In [34]:
import datetime
from unittest import result
from loguru import logger
import argparse
import pickle
from typing import Dict,List
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import global_const as gconst
import copy
import numpy as np
import pickle

# import data


In [35]:
# import 

data_path_prefix = "./esrd"
data_path=f"{data_path_prefix}/processed/"

project_path="./"
result_path=f"{project_path}/result/"
# 656 origin data
pdid_656 = pickle.load(open(f'{data_path}656_pdid','rb'))
x_656 = pickle.load(open(f'{data_path}raw_x_17_656','rb'))
y_656 = pickle.load(open(f'{data_path}656_y','rb'))
static_656 = pickle.load(open(f'{data_path}raw_static_4_656','rb'))


# diet data
diet = pickle.load(open(f'{data_path}diet656_picked.dict','rb'))
from data_diet import get_diet_in
diet_656 = get_diet_in(diet,pdid_656)


date_656 = pickle.load(open(f'{data_path}656_date','rb'))
days_656 = pickle.load(open(f'{data_path}656_days','rb'))

# verify x and y data length
count = 0
for i in range(656):
    if(len(x_656[i])!=len(y_656[i])):
        count += 1

logger.debug(f'verify x and y data length len(x_656), len(y_656)= {len(x_656)}, {len(y_656)}. different count:{count}')

2024-04-05 11:08:53.744 | DEBUG    | __main__:<module>:30 - verify x and y data length len(x_656), len(y_656)= 656, 656. different count:0


# data preprocess

In [36]:
gconst.get_labels_diet()

['water',
 'protein',
 'fat',
 'carbohydrate',
 'Calories',
 'df',
 'k',
 'na',
 'mg',
 'ca',
 'p',
 'fe',
 'zn',
 'cu',
 'mn',
 'se',
 'retinol',
 'vitaminA',
 'carotene',
 'vitaminE',
 'thiamine',
 'riboflavin',
 'nicotinic',
 'ascorbic']

In [37]:
def cal_dpi(protain,kg):
    """cal dpi"""
    return float(protain)/float(kg + 0.0000001)
def cal_dei(calories,kg):
    """cal dei"""
    return float(calories)/float(kg + 0.0000001)

def cal_dpi_ibw(protain,height):
    """cal dpi, use height to calculate ideal body weight"""
    if height < 105:
        height +=100
    weight = height - 105
    
    return float(protain)/(weight + 0.0000001)

def cal_dei_ibw(calories,height):
    """cal dei, use height to calculate ideal body weight"""
    if height < 105:
        height +=100
    weight = height - 105
    return float(calories)/(weight + 0.0000001)

def cal_bmi(height,weight):
    """calculate bmi, height unit cm, weight unit kg"""
    height = height/100
    return float(weight)/(height * height)

def cal_gfr(scr,gender,age):
    """calculate gfr"""
    gfr = 141 * pow(scr,-1.209)*pow(0.993,int(age))
    if gender ==0:
        gfr = gfr*1.018
    return gfr
def cal_age(birthday,day):
    birthday_obj = datetime.datetime.strptime(birthday,'%Y/%m/%d')
    day_obj = datetime.datetime.strptime(day,'%Y/%m/%d')
    age = (day_obj - birthday_obj).days / 365
    return age

## visit data

In [38]:
# concate the dynamic and static data, add bmi and gfr
# recalculate age
labels_visit, labels_static, labels_days = gconst.get_labels_visit(),gconst.get_labels_static(),gconst.get_labels_days()
print(labels_visit)
print(labels_days)
print(labels_days)
res_visit = []
print(len(x_656),len(static_656))
for cur_visits, cur_statics,cur_dates,cur_days in list(zip(x_656,static_656,date_656,days_656)):
    # print(pd.DataFrame(cur_visit,columns=labels_visit))
    # print(pd.DataFrame([cur_statics],columns=labels_static)) 
    # print(cur_days)
    cur_values = []
    for i in range(len(cur_visits)):
        cur_value = list(cur_visits[i])
        # age 
        birthday = cur_days[labels_days.index('birthday')]
        cur_date = cur_dates[i]
        age = cal_age(birthday,cur_date)
        cur_value.append(age)
        cur_value += list(cur_statics[1:])
        # bmi
        height = cur_statics[labels_static.index('height')]
        weight = cur_visits[i][labels_visit.index('Weight')]
        bmi = cal_bmi(height,weight)
        cur_value.append(bmi)
        # mGFR
        scr = cur_visits[i][labels_visit.index('Scr')]
        gender = cur_statics[labels_static.index('gender')]
        gfr = cal_gfr(scr,gender,age)
        cur_value.append(gfr)
        cur_values.append(cur_value)
    res_visit.append(cur_values)
labels_statistics_visit = labels_visit + labels_static + ['bmi','gfr']

with open(f'{data_path}statistics_visit','wb') as f:
    pickle.dump(res_visit,f)



['Cl', 'CO2CP', 'WBC', 'Hb', 'Urea', 'Ca', 'K', 'Na', 'Scr', 'P', 'Albumin', 'hs-CRP', 'Glucose', 'Appetite', 'Weight', 'SBP', 'DBP']
['birthday', 'admission_day', 'age', 'death', 'death_day', 'death_age']
['birthday', 'admission_day', 'age', 'death', 'death_day', 'death_age']
656 656


In [39]:
# 初次就诊
res_visit_first = [i[0] for i in res_visit]
print(len(res_visit_first))
with open(f'{data_path}statistics_visit_first','wb') as f:
    pickle.dump(res_visit_first,f)

656


In [40]:
len(labels_statistics_visit)

23

In [41]:
res_visit_mean = []
for cur_visit in res_visit:
    cur_visit_np = np.array(cur_visit)
    cur_visit_np_nodate = np.array(cur_visit_np,float) 
    cur_visit_mean = np.mean(cur_visit_np_nodate,axis=0)
    # print(cur_diet_mean.shape)
    cur_value = list(cur_visit_mean)
    res_visit_mean.append(cur_value)

print(np.array(res_visit_mean).shape)
with open(f'{data_path}statistics_visit_mean','wb') as f:
    pickle.dump(res_visit_mean,f)


(656, 23)


## diet data

In [42]:

labels_diet = gconst.get_labels_diet()
print(labels_diet)
labels_statistics_diet = ['date'] + labels_diet + ['dpi']

from data_diet import diet_dict2matrix,diet_dict2dict,get_diet_visit_align_index
diet_matrixs = []
for id in pdid_656:
    diet_matrix = diet_dict2matrix({id:diet_656[id]},no_id=True,no_date=False)
    diet_matrixs.append(diet_matrix)

diet_visit_align = get_diet_visit_align_index(pdid_656,diet_656,date_656)

res_diet = []
for cur_visits,cur_diets,cur_align,cur_statics in list(zip(x_656,diet_matrixs,diet_visit_align,static_656)):
    cur_values = []
    for i in range(len(cur_diets)):
        cur_value = list(cur_diets[i])
        cur_align_index = cur_align[i]
        weight = cur_visits[cur_align_index][labels_visit.index("Weight")]
        protein  = cur_diets[i][labels_statistics_diet.index("protein")]
        calories  = cur_diets[i][labels_statistics_diet.index("Calories")]
        height = cur_statics[labels_static.index("height")]
        # dpi = cal_dpi(protein,weight)
        # dei = cal_dei(calories,weight)
        dpi_ibw = cal_dpi_ibw(protein,height)
        dei_ibw = cal_dei_ibw(calories,height)
        dpi = dpi_ibw
        dei = dei_ibw
        cur_value.append(dpi)
        cur_value.append(dei)
        cur_values.append(cur_value)
    res_diet.append(cur_values)

with open(f'{data_path}statistics_diet','wb') as f:
    pickle.dump(res_diet,f)


['water', 'protein', 'fat', 'carbohydrate', 'Calories', 'df', 'k', 'na', 'mg', 'ca', 'p', 'fe', 'zn', 'cu', 'mn', 'se', 'retinol', 'vitaminA', 'carotene', 'vitaminE', 'thiamine', 'riboflavin', 'nicotinic', 'ascorbic']


In [43]:
res_diet_first = [i[0] for i in res_diet]
print(len(res_visit_first))
with open(f'{data_path}statistics_diet_first','wb') as f:
    pickle.dump(res_diet_first,f)

656


In [44]:
res_diet_mean = []
for cur_diet in res_diet:
    cur_diet_np = np.array(cur_diet)
    cur_diet_np_nodate = np.array(cur_diet_np[:,1:],float)
    cur_diet_mean = np.mean(cur_diet_np_nodate,axis=0)
    # print(cur_diet_mean.shape)
    cur_value = ['mean'] + list(cur_diet_mean)
    res_diet_mean.append(cur_value)

print(np.array(res_diet_mean).shape)
with open(f'{data_path}statistics_diet_mean','wb') as f:
    pickle.dump(res_diet_mean,f)


(656, 27)
